<a href="https://colab.research.google.com/github/curiosity806/2020_dacon_satellite_precipitation/blob/master/Dacon_satellite_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>